In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Text preprocessing . Added as tokenizer in Count_Vectoriser()
def text_process(mess): 
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Lemmatize words
    4. Returns the modified string . 
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [wnl.lemmatize(word) for word in nopunc.split() if word.lower() 
    not in stopwords.words('english')]
  

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
nltk.download('stopwords')
nltk.download('wordnet')
wnl=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project 2/train.csv')

df_sm = df.sample(frac=1)#use fraction of data for debugging, (the data is huge)
x = df_sm['body']
y = df_sm['subreddit']
xtrain,xtest,ytrain,ytest =train_test_split(x,y, test_size=0.33,random_state=42)





In [ ]:
#create dictionary with the data and binary word vectors 
cv=CountVectorizer(tokenizer=text_process,binary=True,ngram_range=(1,1),
                   max_features=1000) 
train_vectors=cv.fit_transform(xtrain)
test_vectors=cv.transform(xtest)
df_train=pd.DataFrame.sparse.from_spmatrix(train_vectors,index=ytrain)

In [ ]:
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
subreddit,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hardware,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hardware,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
gamedev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
datascience,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
datascience,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rpg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
rpg,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
gamernews,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#calculate theta_k for every class
theta_k= {k:(ytrain == k).sum()/float(ytrain.shape[0]) for k in ytrain.unique()} 

theta_k_j = {k: 
             
             {j: ( df_train.loc[k][j].sum() +1) / float( (ytrain ==k).sum() + 2) 
             for j in df_train.columns } #calculate theta_k_j for every k
             
             for k in ytrain.unique()}

#create dataframe for easier vector multplication
df_theta_kj=pd.DataFrame.from_dict(theta_k_j,orient='index') 

In [ ]:
dict_prob={k:(test_vectors @ np.log(df_theta_kj.loc[k]) + 
              (1-test_vectors.toarray()) @ np.log(1-df_theta_kj.loc[k]) + 
              theta_k[k] ) for k in ytrain.unique()}
#create dataframe with class probability for every sample
class_prob = pd.DataFrame.from_dict(dict_prob,orient='columns') 

In [ ]:
#extract the class with max probability for each sample
predictions = class_prob.idxmax(axis=1) 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
print( classification_report(ytest,predictions))
print('\n',accuracy_score(ytest,predictions))

              precision    recall  f1-score   support

       anime       0.96      0.66      0.78       449
        cars       0.85      0.85      0.85       663
   computers       0.54      0.69      0.61       141
 datascience       0.95      0.83      0.88       807
     gamedev       0.82      0.54      0.65       334
   gamernews       0.28      0.81      0.41       252
    hardware       0.79      0.73      0.76       531
         rpg       0.86      0.71      0.78       646

    accuracy                           0.75      3823
   macro avg       0.76      0.73      0.72      3823
weighted avg       0.83      0.75      0.77      3823


 0.746534135495684


# **KFold cross validation**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
kf =KFold(n_splits=5,shuffle=True)
accuracy_kfold = []
for train, test in kf.split(x):

  #create dictionary with the data and binary word vectors 
  cv=CountVectorizer(binary=True,ngram_range=(1,1),
                     max_features=1000) 
  train_vectors=cv.fit_transform(x[train])
  test_vectors=cv.transform(x[test])
  df_train=pd.DataFrame.sparse.from_spmatrix(train_vectors,index=y[train])

  #calculate theta_k for every class
  theta_k= {k:(y[train] == k).sum()/float(y[train].shape[0]) for 
            k in y[train].unique()} 

  theta_k_j = {k: 
             
              {j: ( df_train.loc[k][j].sum() +1) /float( (y[train] ==k).sum()+2) 
              for j in df_train.columns } #calculate theta_k_j for every k
             
              for k in y[train].unique()}

  #create dataframe for easier vector multplication
  df_theta_kj=pd.DataFrame.from_dict(theta_k_j,orient='index') 

  dict_prob={k:(test_vectors @ np.log(df_theta_kj.loc[k]) + 
                (1-test_vectors.toarray()) @ np.log(1-df_theta_kj.loc[k]) + 
                theta_k[k] ) for k in ytrain.unique()}
  #create dataframe with class probability for every sample
  class_prob = pd.DataFrame.from_dict(dict_prob,orient='columns') 

  #extract the class with max probability for each sample
  predictions = class_prob.idxmax(axis=1) 
  accuracy_kfold.append(accuracy_score(y[test],predictions))

In [ ]:
from statistics import mean
print(mean(accuracy_kfold))

0.1924543603894918


In [ ]:
#test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project 2/test.csv')
#test= test_data['body']
#corpus = pd.concat([x,test],axis=0)